In [3]:
import torch
import torch.nn as nn
from networks import DynamicUNet, AdditionalLayers

network = DynamicUNet(1, 3, [64, 128])

additional_pass = AdditionalLayers(128, 256)

network.catenate_models(additional_pass)

loss = network.calculate_train_loss(torch.randn((5, 1, 64, 64)), torch.ones((5, 1, 64, 64)), alpha=0.5, loss_fn = nn.MSELoss())

In [4]:
from torchviz import make_dot

make_dot(loss, params=dict(list(network.named_parameters()))).render("torchviz3", format="png")

'torchviz3.png'